In [7]:
import pandas as pd
import sqlite3

# Create a connection to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# --- Load each CSV file into a pandas DataFrame ---
orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')

# --- Write each DataFrame to a new table in our SQL database ---
orders.to_sql('orders', conn, index=False, if_exists='replace')
order_items.to_sql('order_items', conn, index=False, if_exists='replace')
products.to_sql('products', conn, index=False, if_exists='replace')
customers.to_sql('customers', conn, index=False, if_exists='replace')
payments.to_sql('payments', conn, index=False, if_exists='replace')

print("Database created and all tables loaded successfully.")

# Helper function to run queries and return the result as a DataFrame
def run_query(query):
  return pd.read_sql_query(query, conn)

# Let's list the tables to confirm they were created
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
print("\nTables in the database:")
run_query(query_tables)
import pandas as pd


Database created and all tables loaded successfully.

Tables in the database:


In [8]:
query1 = """
SELECT
  order_status,
  COUNT(order_id) AS NumberOfOrders
FROM
  orders
GROUP BY
  order_status
ORDER BY
  NumberOfOrders DESC;
"""
print("--- Number of Orders by Status ---")
run_query(query1)

--- Number of Orders by Status ---


,order_status,NumberOfOrders
0,delivered,5734
1,shipped,70
2,canceled,35
3,unavailable,30
4,processing,21
5,invoiced,14


In [9]:
query2 = """
SELECT
  p.product_category_name,
  COUNT(oi.order_item_id) AS ItemsSold
FROM
  order_items oi
JOIN
  products p ON oi.product_id = p.product_id
GROUP BY
  p.product_category_name
ORDER BY
  ItemsSold DESC
LIMIT 10;
"""
print("--- Top 10 Product Categories by Items Sold ---")
run_query(query2)

--- Top 10 Product Categories by Items Sold ---


,product_category_name,ItemsSold
0,cama_mesa_banho,2702
1,beleza_saude,2500
2,moveis_decoracao,2123
3,esporte_lazer,1959
4,informatica_acessorios,1872
5,utilidades_domesticas,1775
6,relogios_presentes,1357
7,ferramentas_jardim,1134
8,telefonia,1103
9,automotivo,998


In [10]:
query3 = """
SELECT
  strftime('%Y-%m', o.order_purchase_timestamp) AS SalesMonth,
  SUM(p.payment_value) AS TotalRevenue
FROM
  orders o
JOIN
  payments p ON o.order_id = p.order_id
WHERE
  strftime('%Y', o.order_purchase_timestamp) = '2017'
  AND o.order_status = 'delivered'
GROUP BY
  SalesMonth
ORDER BY
  SalesMonth;
"""
print("--- Total Revenue per Month in 2017 ---")
run_query(query3)

--- Total Revenue per Month in 2017 ---


,SalesMonth,TotalRevenue
0,2017-01,8515.48
1,2017-02,15300.24
2,2017-03,21443.72
3,2017-04,19140.31
4,2017-05,41206.97
5,2017-06,22108.31
6,2017-07,36329.74
7,2017-08,37253.67
8,2017-09,37401.97
9,2017-10,39067.55


In [11]:
query4 = """
SELECT
  c.customer_city,
  SUM(p.payment_value) AS TotalRevenue
FROM
  customers c
JOIN
  orders o ON c.customer_id = o.customer_id
JOIN
  payments p ON o.order_id = p.order_id
WHERE
  o.order_status = 'delivered'
GROUP BY
  c.customer_city
ORDER BY
  TotalRevenue DESC
LIMIT 5;
"""
print("--- Top 5 Cities by Revenue ---")
run_query(query4)

--- Top 5 Cities by Revenue ---


,customer_city,TotalRevenue
0,sao paulo,126906.14
1,rio de janeiro,76955.47
2,brasilia,19783.21
3,belo horizonte,17637.05
4,curitiba,15491.69
